In [3]:
!pip install pycryptodome

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 8.0 MB/s eta 0:00:00


In [4]:
from Crypto.Util.number import getPrime, inverse, bytes_to_long, getRandomRange
from Crypto.Random import get_random_bytes
import random
from random import randrange, shuffle

In [5]:
def secure_random_number(p):
    while True:
        random_bytes = get_random_bytes((p.bit_length() + 7) // 8)
        random_number = bytes_to_long(random_bytes) % (p - 1) + 1
        return random_number

In [6]:
# ElGamal key generation, encryption, and decryption functions

In [7]:
def generate_keys(bit_length=256):
    p = getPrime(bit_length)
    g = 2
    x = getRandomRange(2, p - 1)
    h = pow(g, x, p)
    return (p, g, h), x

In [8]:
def encrypt_vote(public_key, vote_choice):
    p, g, h = public_key
    k = getRandomRange(2, p - 1)
    c1 = pow(g, k, p)
    c2 = (vote_choice * pow(h, k, p)) % p
    return c1, c2

In [9]:
def decrypt_vote(private_key, ciphertext, p):
    c1, c2 = ciphertext
    s = pow(c1, private_key, p)
    m = (c2 * inverse(s, p)) % p
    return m

In [10]:
# Mixing and Re-Encryption functions implementing Neff's scheme

In [11]:
def mix_and_reencrypt_votes(public_key, encrypted_votes):
    p, g, h = public_key
    shuffled_votes = encrypted_votes.copy()
    shuffle(shuffled_votes)

    # Re-encrypting votes using a new random exponent k' for each vote
    re_encrypted_votes = []
    for c1, c2 in shuffled_votes:
        k_prime = getRandomRange(2, p - 1)
        c1_prime = (c1 * pow(g, k_prime, p)) % p
        c2_prime = (c2 * pow(h, k_prime, p)) % p
        re_encrypted_votes.append((c1_prime, c2_prime))

    return re_encrypted_votes

In [12]:
# Neff's scheme functions for zero-knowledge proof of shuffle

In [13]:
def generate_permutation(k):
    """Generate a random permutation of k elements."""
    permutation = list(range(1, k + 1))
    random.shuffle(permutation)
    return permutation

In [14]:
def generate_sequences(k, p, g):
    """Generate sequences of k elements from Zq-{0}, and a random exponent d."""
    e1 = [secure_random_number(p) for _ in range(k)]
    e2 = [secure_random_number(p) for _ in range(k)]

    E1 = [pow(g, e, p) for e in e1]
    E2 = [pow(g, e, p) for e in e2]

    # Verifier generates f1 and f2 sequences
    f1 = [secure_random_number(p) for _ in range(k)]
    f2 = [secure_random_number(p) for _ in range(k)]

    return e1, e2, E1, E2, f1, f2

In [15]:
def compute_F_sequences(k, p, g, d, e1, e2, f1, f2, permutation):
    """Compute the sequences F1 and F2 using the permutation and values d, e1, e2, f1, f2."""
    F1 = []
    F2 = []

    # Computing the permutation inverse
    pi_inv = [0] * k
    for i in range(k):
        pi_inv[permutation[i] - 1] = i

    # Computing F1i and F2i
    for i in range(k):
        F1.append(pow(g, d * f1[pi_inv[i]] * e1[pi_inv[i]], p))
        F2.append(pow(g, d * f2[pi_inv[i]] * e2[pi_inv[i]], p))

    return F1, F2, pi_inv

In [16]:
def compute_U_V_D_sequences(k, p, g, d, e1, e2, f1, f2, gamma, pi_inv):
    """Compute the U, V sequences and D value for the k-shuffle protocol."""
    U = []
    V = []

    # Computing Ui and Vi values
    for i in range(k):
        # Calculating indices from the permutation inverse
        pi_i = pi_inv[i]

        U.append(pow(g, (f1[pi_i] * e1[pi_i] + gamma * f2[pi_i] * e2[pi_i]) % (p - 1), p))
        V.append(pow(g, (d * f1[pi_i] + gamma * d * f2[pi_i]) % (p - 1), p))

    D = pow(g, d, p)

    return U, V, D

In [17]:
def compute_A_B_sequences(k, p, g, U, V, d, pi_inv):
    """Generate the a_i, b_i sequences and compute A_i = g^a_i, B_i = g^b_i."""
    # Generating random a_i and b_i sequences
    a = [secure_random_number(p) for _ in range(k)]
    b = [secure_random_number(p) for _ in range(k-1)]

    # Calculating b_k as specified in the protocol
    sum_a_ui = sum(a[i] * pi_inv[i] for i in range(k-1))  # Assuming pi_inv is u
    sum_b_vi = sum(b[i] * pi_inv[i] for i in range(k-1))  # Assuming pi_inv is v
    b_k = inverse(V[-1], p) * (sum_b_vi - d * sum_a_ui) % (p - 1)
    b.append(b_k)  # Append b_k to the b sequence

    # Computing A_i = g^a_i and B_i = g^b_i
    A = [pow(g, a_i, p) for a_i in a]
    B = [pow(g, b_i, p) for b_i in b]

    return A, B, a, b

In [18]:
def generate_lambda(p):
    """Generate a new random challenge lambda from Zq."""
    lambda_ = secure_random_number(p)
    return lambda_

def generate_random_challenge(p):
    """Generate a random challenge gamma from Zq - {0}."""
    gamma = secure_random_number(p)
    return gamma

def compute_exponents_and_quantities(k, p, g, X, Y, a, b, u, v, lambda_):
    """Compute the exponents s_i, r_i and the quantities P, Q."""
    s = [(a[i] + lambda_ * u[i]) % (p - 1) for i in range(k)]
    r = [(b[i] + lambda_ * v[i]) % (p - 1) for i in range(k)]

    # Compute quantities P and Q
    P = 1
    Q = 1
    for i in range(k):
        P = (P * pow(X[i], r[i], p)) % p
        Q = (Q * pow(Y[i], s[i], p)) % p

    return s, r, P, Q

In [19]:
# step 9
def generate_commitments(U, V, D, g, p):
    """Generate cryptographic commitments for U, V, and D values."""
    commitments_U = [pow(g, u, p) for u in U]
    commitments_V = [pow(g, v, p) for v in V]
    commitment_D = pow(g, D, p)
    return commitments_U, commitments_V, commitment_D

def generate_silmpp_responses(U, V, D, lambda_, p):
    """Generate responses for SILMPP based on a challenge lambda."""
    responses_U = [(u + lambda_) % p for u in U]
    responses_V = [(v + lambda_) % p for v in V]
    response_D = (D + lambda_) % p
    return responses_U, responses_V, response_D

def verify_silmpp(g, p, commitments, responses, challenge):
    """Verify the SILMPP by recalculating and comparing the commitment from responses."""
    verified = True
    for i, (commitment, response) in enumerate(zip(commitments, responses)):
        computed_commitment = pow(g, response, p) * pow(inverse(g, p), challenge, p) % p
        if computed_commitment != commitment:
#             print(f"Verification failed for commitment {i}: expected {commitment}, got {computed_commitment}")
            verified = False
#         else:
#             print(f"Verification Sucess for commitment {i}: expected {commitment}, got {computed_commitment}")

    print(" ")

    return verified


In [20]:
# Additional functions for tallying votes and verifying the mixing

In [21]:
# Talling decrypted votes
def talling(decrypted_votes):
    tally = {}
    for vote in decrypted_votes:
        if vote in tally:
            tally[vote] += 1
        else:
            tally[vote] = 1

    max_votes = max(tally.values())

    winners = [candidate for candidate, votes in tally.items() if votes == max_votes]

    if len(winners) == 1:
        print(f"Winner is Candidate {winners[0]} with {max_votes} votes.")
    else:
        winners_str = ", ".join(map(str, winners))
        print(f"It's a draw between Candidates {winners_str} with {max_votes} votes each.")


In [22]:
# Setup for Neff's ZKP
def ZKP(mixed_votes, p, g, h, public_key, private_key, mixnet_number):

    permutation = generate_permutation(len(mixed_votes))
    e1, e2, E1, E2, f1, f2 = generate_sequences(len(mixed_votes), p, g)
    F1, F2, pi_inv = compute_F_sequences(len(mixed_votes), p, g, private_key, e1, e2, f1, f2, permutation)
    gamma = generate_random_challenge(p)
    U, V, D = compute_U_V_D_sequences(len(mixed_votes), p, g, private_key, e1, e2, f1, f2, gamma, pi_inv)
    A, B, a, b = compute_A_B_sequences(len(mixed_votes), p, g, U, V, private_key, pi_inv)

    lambda_ = generate_lambda(p)
    s, r, P, Q = compute_exponents_and_quantities(len(mixed_votes), p, g, U, V, a, b, e1, f1, lambda_)

    # Generate commitments for SILMPP
    commitments_U = [pow(g, u, p) for u in U]
    commitments_V = [pow(g, v, p) for v in V]
    commitment_D = pow(g, D, p)

    # Generate responses for SILMPP
    responses_U = [(u + lambda_) % (p - 1) for u in U]
    responses_V = [(v + lambda_) % (p - 1) for v in V]
    response_D = (D + lambda_) % (p - 1)

    # Verify SILMPP
    if verify_silmpp(g, p, commitments_U + commitments_V + [commitment_D], responses_U + responses_V + [response_D], lambda_):
        print(f"SILMPP verification successful after {mixnet_number}st shuffle")
        print(f"Votes have not been altered or manipulated in {mixnet_number}st mixnet server")
        return True
    else:
        print(f"SILMPP verification failed after {mixnet_number}st shuffle")
        print(f"Votes have been altered or manipulated in {mixnet_number}st mixnet server")
        return False

In [23]:
def ZKP_check(Mixers_votes,original_votes, p, g, h, public_key, private_key, mixnet_number):

    permutation = generate_permutation(len(original_votes))
    permutation_altered = generate_permutation(len(Mixers_votes))

    e1, e2, E1, E2, f1, f2 = generate_sequences(len(original_votes), p, g)
    F1, F2, pi_inv = compute_F_sequences(len(original_votes), p, g, private_key, e1, e2, f1, f2, permutation)

    e1_altered, e2_altered, E1_altered, E2_altered, f1_altered, f2_altered = generate_sequences(len(Mixers_votes), p, g)
    F1_altered, F2_altered, pi_inv_altered = compute_F_sequences(len(Mixers_votes), p, g, private_key, e1_altered, e2_altered, f1_altered, f2_altered, permutation_altered)

    gamma = generate_random_challenge(p)
    lambda_ = generate_lambda(p)

    U, V, D = compute_U_V_D_sequences(len(original_votes), p, g, private_key, e1, e2, f1, f2, gamma, pi_inv)
    A, B, a, b = compute_A_B_sequences(len(original_votes), p, g, U, V, private_key, pi_inv)
    s, r, P, Q = compute_exponents_and_quantities(len(original_votes), p, g, U, V, a, b, e1, f1, lambda_)

    U_altered, V_altered, D_altered = compute_U_V_D_sequences(len(Mixers_votes), p, g, private_key, e1_altered, e2_altered, f1_altered, f2_altered, gamma, pi_inv_altered)
    A_altered, B_altered, a_altered, b_altered = compute_A_B_sequences(len(Mixers_votes), p, g, U_altered, V_altered, private_key, pi_inv_altered)
    s_altered, r_altered, P_altered, Q_altered = compute_exponents_and_quantities(len(Mixers_votes), p, g, U_altered, V_altered, a_altered, b_altered, e1_altered, f1_altered, lambda_)

    # Generate commitments for SILMPP
    commitments_U = [pow(g, u, p) for u in U]
    commitments_V = [pow(g, v, p) for v in V]
    commitment_D = pow(g, D, p)

    # Generate responses for SILMPP
    responses_U_altered = [(u_altered + lambda_) % (p - 1) for u_altered in U_altered]
    responses_V_altered = [(v_altered + lambda_) % (p - 1) for v_altered in V_altered]
    response_D_altered = (D_altered + lambda_) % (p - 1)

    # Verify SILMPP
    if verify_silmpp(g, p, commitments_U + commitments_V + [commitment_D], responses_U_altered + responses_V_altered + [response_D_altered], lambda_):
        print(f"SILMPP verification successful after {mixnet_number}st shuffle")
        print(f"Votes have not been altered or manipulated in {mixnet_number}st mixnet server")
        return True
    else:
        print(f"SILMPP verification failed after {mixnet_number}st shuffle")
        print(f"Votes have been altered or manipulated in {mixnet_number}st mixnet server")
        return False

In [24]:
def main():
    # Generate public and private keys
    public_key, private_key = generate_keys(256)
    print(f"Public Key: {public_key}")
    p, g, h = public_key

    # Get number of voters and their votes
    num_votes = int(input("Enter the number of voters: "))
    votes = []
    for i in range(num_votes):
        vote = int(input(f"Enter vote for voter {i+1} (use integer values for candidates): "))
        votes.append(vote)

    encrypted_original_votes = [encrypt_vote(public_key, vote) for vote in votes]
    print("\nEncrypted Original Votes:", encrypted_original_votes)
    # Perform mixing and re-encryption
    for i in range(3):  # Simulating 3 mixnet servers.
        mixed_original_votes = mix_and_reencrypt_votes(public_key, encrypted_original_votes)

        #alternation Simulation
        # mixed_Mixers_votes = [vote[0] + 1 for vote in encrypted_original_votes]
        mixed_Mixers_votes = encrypted_original_votes # commnet this line if altering the votes

        if(mixed_Mixers_votes == encrypted_original_votes) :
            ZKP_Sucess = ZKP(mixed_original_votes, p, g, h, public_key, private_key, i+1)#checking if the votes are manipulated after each mixing
            if ZKP_Sucess==False :
                break;

        else :
            ZKP_Sucess = ZKP_check(mixed_Mixers_votes, mixed_original_votes, p, g, h, public_key, private_key, i+1)#checking if the votes are manipulated after each mixing
            if ZKP_Sucess==False :
                break;

    if ZKP_Sucess==True :
        print("")
        print("Mixed and Re-Encrypted Votes:", mixed_Mixers_votes)
        print("")
        decrypted_votes = [decrypt_vote(private_key, vote, p) for vote in mixed_Mixers_votes]
        talling(decrypted_votes)


In [25]:
if __name__ == "__main__" :
    main()

Public Key: (64267670887845697656919676718984422488898732919124113859985939653948802332733, 2, 28155697036404075047632583612341301919862932197109084997173065146510023802448)
Enter the number of voters: 5
Enter vote for voter 1 (use integer values for candidates): 1
Enter vote for voter 2 (use integer values for candidates): 1
Enter vote for voter 3 (use integer values for candidates): 1
Enter vote for voter 4 (use integer values for candidates): 2
Enter vote for voter 5 (use integer values for candidates): 2

Encrypted Original Votes: [(38186300029851686923614551338041736832472659852439109054360601468756690620768, 34425675647368341527166700324000771978021375260887071664986877682811353714088), (56809624031649764597707815325971818460050408980310327117009971686339287150267, 62917477458400634524389230463934021949662832916393089051063979291377473930019), (49622133444930984259654922005172050228536959436623517013867988264081459456616, 4841652841409844023935163678260009051656103756172334561688